In [11]:
import os 
import sys
import re 
import numpy as np
import agilent_loader as ag
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import warnings

folder = "26-12-2022\\Location1"

results_folder = folder+"\\Results"

def fit_image_to_polynomial(image, degree):
	# Step 1 & 2: Flatten the 2D array and generate meshgrid for coordinates
	y, x = np.indices(image.shape)
	x = x.flatten()
	y = y.flatten()
	z = image.flatten()  # Flatten the image array

	# Polynomial model function
	def poly_model(coeffs, x, y, degree, z=None):
		idx = 0
		model = np.zeros_like(x, dtype=np.float64)
		for i in range(degree + 1):
			for j in range(degree + 1 - i):
				model += coeffs[idx] * (x ** i) * (y ** j)
				idx += 1
		return model - z if z is not None else model

	# Number of coefficients for a given degree
	def num_coeffs(degree):
		return (degree + 1) * (degree + 2) // 2

	# Step 4: Fit polynomial
	initial_guess = np.zeros(num_coeffs(degree))
	res = least_squares(poly_model, initial_guess, args=(x, y, degree, z))

	# Step 5: Evaluate polynomial
	fitted_image = poly_model(res.x, x, y, degree).reshape(image.shape)

	return fitted_image, res.x

def check_and_prepare_folder(folder_path):
    # Check if the folder exists
    if not os.path.exists(folder_path):
        # If the folder does not exist, create it
        os.makedirs(folder_path)
        print(f"Folder '{folder_path}' created.")
    else:
        # If the folder exists, ensure it is empty
        for filename in os.listdir(folder_path):
            file_path = os.path.join(folder_path, filename)
            try:
                if os.path.isfile(file_path) or os.path.islink(file_path):
                    os.unlink(file_path)  # Remove file or link
                elif os.path.isdir(file_path):
                    shutil.rmtree(file_path)  # Remove directory
            except Exception as e:
                print(f'Failed to delete {file_path}. Reason: {e}')
        print(f"Folder '{folder_path}' already exists and is now empty.")

check_and_prepare_folder(results_folder)

files_in_folder = [f for f in os.listdir(folder) if '.mi' in f]


for filename in files_in_folder:
	# Generate the new filename by replacing spaces with underscores and removing "@"
	new_filename = filename.replace(" ", "_").replace("@", "")
	
	# Construct the full old and new file paths
	old_file_path = os.path.join(folder, filename)
	new_file_path = os.path.join(folder, new_filename)
	
	# Rename the file
	os.rename(old_file_path, new_file_path)

files_in_folder = [f for f in os.listdir(folder) if '.mi' in f]

print(f" Found {len(files_in_folder)} files in {folder}")
print(files_in_folder)

Folder '26-12-2022\Location1\Results' already exists and is now empty.
 Found 500 files in 26-12-2022\Location1
['lls_00000.mi', 'lls_00001.mi', 'lls_00002.mi', 'lls_00003.mi', 'lls_00004.mi', 'lls_00005.mi', 'lls_00006.mi', 'lls_00007.mi', 'lls_00008.mi', 'lls_00009.mi', 'lls_00010.mi', 'lls_00011.mi', 'lls_00012.mi', 'lls_00013.mi', 'lls_00014.mi', 'lls_00015.mi', 'lls_00016.mi', 'lls_00017.mi', 'lls_00018.mi', 'lls_00019.mi', 'lls_00020.mi', 'lls_00021.mi', 'lls_00022.mi', 'lls_00023.mi', 'lls_00024.mi', 'lls_00025.mi', 'lls_00026.mi', 'lls_00027.mi', 'lls_00028.mi', 'lls_00029.mi', 'lls_00030.mi', 'lls_00031.mi', 'lls_00032.mi', 'lls_00033.mi', 'lls_00034.mi', 'lls_00035.mi', 'lls_00036.mi', 'lls_00037.mi', 'lls_00038.mi', 'lls_00039.mi', 'lls_00040.mi', 'lls_00041.mi', 'lls_00042.mi', 'lls_00043.mi', 'lls_00044.mi', 'lls_00045.mi', 'lls_00046.mi', 'lls_00047.mi', 'lls_00048.mi', 'lls_00049.mi', 'lls_00050.mi', 'lls_00051.mi', 'lls_00052.mi', 'lls_00053.mi', 'lls_00054.mi', 'lls_00

In [12]:
Cof_for_runs=np.zeros(len(files_in_folder))
Cof_for_runs_std=np.zeros(len(files_in_folder))
ii=0


for file_path in files_in_folder:
	
	file=ag.load_mi(folder+"\\"+file_path)	
	friction_arrays=[]
	topography_arrays=[]
	print(file_path)
	for buffer in file.buffers:
		
		if buffer.bufferLabel == "Friction":
			friction_arrays.append(buffer.data)
			
		elif buffer.bufferLabel == "Topography":
			topography_arrays.append(buffer.data)

	print(len(friction_arrays),len(topography_arrays))

			
	if len(friction_arrays) == 2:
		averaged_friction = ((friction_arrays[1]) - (friction_arrays[0])) / 2

		friction_std=np.std(averaged_friction)
		friction_mean=np.mean(averaged_friction)
		Cof_for_runs[ii]=friction_mean
		Cof_for_runs_std[ii]=friction_std

	else:
		warnings.warn(f"{file_path} doesnt contain both trace and retrace friction chunks\nExpected 2 friction arrays, got %d" % len(friction_arrays)+f" with file {file_path}")
		# raise ValueError("Expected 2 friction arrays, got %d" % len(friction_arrays)+f" with file {file_path} ")
		continue

	if (ii==0):
		

		fig,ax=plt.subplots(1,2,figsize=(10,5),dpi=300)
		# file.extent=file.extent*1e6
		# print(f"extent {file.extent}")
		file.extent[1]=file.extent[1]-file.extent[0]
		file.extent[0]=0
		file.extent[3]=file.extent[3]-file.extent[2]
		file.extent[2]=0
		# print(f"extent {file.extent}")
		fit_topology, _ = fit_image_to_polynomial(topography_arrays[0], 2)

		im1=ax[0].imshow(averaged_friction,cmap='inferno', extent=file.extent)
		im2=ax[1].imshow(topography_arrays[0]*1e9,cmap='inferno', extent=file.extent)
		ax[0].set_title(file_path+" friction force")


		ax[0].set_title("Friction force")
		ax[1].set_title("Topography")
		ax[0].set_xticks([])
		ax[0].set_yticks([])
		ax[1].set_xticks([])
		ax[1].set_yticks([])


		scale_length_um = 1
		x_pad=0.9
		y_pad=0.1
		x_low=file.extent[1]*x_pad
		y_low=file.extent[3]*y_pad
	
		scale_bar1 = Line2D([x_low, x_low+ scale_length_um], [y_low,y_low], color='white', linewidth=3)
		scale_bar2 = Line2D([x_low, x_low+ scale_length_um], [y_low,y_low], color='white', linewidth=3)
		
		
		ax[0].add_line(scale_bar1)
		ax[0].text(x_low+ scale_length_um/2, y_low, f'{scale_length_um} $\mu m$', color='white', ha='center', va='bottom')
		ax[1].add_line(scale_bar2)
		ax[1].text(x_low+ scale_length_um/2, y_low, f'{scale_length_um} $\mu m$', color='white', ha='center', va='bottom')

		cbar1=fig.colorbar(im1,ax=ax[0],fraction=0.046, pad=0.04)
		cbar2=fig.colorbar(im2,ax=ax[1],fraction=0.046, pad=0.04)
		cbar1.set_label("Friction force [V]")
		cbar2.set_label("Height $[ nm]$")

		plt.tight_layout(rect=[0, 0, 1, 0.95])
		plt.savefig(results_folder+f"\\Friction_force_and_topography_{file_path}.png")
		plt.clf()
		plt.close()

	ii+=1


plt.figure(figsize=(10,5),dpi=300)
plt.plot(np.arange(len(Cof_for_runs)),Cof_for_runs)	
plt.fill_between(np.arange(len(Cof_for_runs)),Cof_for_runs-Cof_for_runs_std,Cof_for_runs+Cof_for_runs_std,alpha=0.5)
plt.title("Friction force as a function of cycles")
plt.xlabel("Cycles over the sample")
plt.ylabel("Friction force [V]")
plt.savefig(results_folder+"\\Friction_force_for_cycles.png")


# plt.xlim(len(Cof_for_runs)/5,len(Cof_for_runs)/4)
# plt.ylim(2,4)



lls_00000.mi
2 2


C:\Users\cordo\AppData\Local\Temp\ipykernel_33644\2822945733.py:80: UserWarning: Tight layout not applied. The left and right margins cannot be made large enough to accommodate all axes decorations.
  plt.tight_layout(rect=[0, 0, 1, 0.95])


lls_00001.mi
2 2
lls_00002.mi
2 2
lls_00003.mi
2 2
lls_00004.mi
2 2
lls_00005.mi
2 2
lls_00006.mi
2 2
lls_00007.mi
2 2
lls_00008.mi
2 2
lls_00009.mi
2 2
lls_00010.mi
2 2
lls_00011.mi
2 2
lls_00012.mi
2 2
lls_00013.mi
2 2
lls_00014.mi
2 2
lls_00015.mi
2 2
lls_00016.mi
2 2
lls_00017.mi
2 2
lls_00018.mi
2 2
lls_00019.mi
2 2
lls_00020.mi
2 2
lls_00021.mi
2 2
lls_00022.mi
2 2
lls_00023.mi
2 2
lls_00024.mi
2 2
lls_00025.mi
2 2
lls_00026.mi
2 2
lls_00027.mi
2 2
lls_00028.mi
2 2
lls_00029.mi
2 2
lls_00030.mi
2 2
lls_00031.mi
2 2
lls_00032.mi
2 2
lls_00033.mi
2 2
lls_00034.mi
2 2
lls_00035.mi
2 2
lls_00036.mi
2 2
lls_00037.mi
2 2
lls_00038.mi
2 2
lls_00039.mi
2 2
lls_00040.mi
2 2
lls_00041.mi
2 2
lls_00042.mi
2 2
lls_00043.mi
2 2
lls_00044.mi
2 2
lls_00045.mi
2 2
lls_00046.mi
2 2
lls_00047.mi
2 2
lls_00048.mi
2 2
lls_00049.mi
2 2
lls_00050.mi
2 2
lls_00051.mi
2 2
lls_00052.mi
2 2
lls_00053.mi
2 2
lls_00054.mi
2 2
lls_00055.mi
2 2
lls_00056.mi
2 2
lls_00057.mi
2 2
lls_00058.mi
2 2
lls_00059.mi
2

KeyboardInterrupt: 

ValueError: Image size of 96595432x1100 pixels is too large. It must be less than 2^16 in each direction.

<Figure size 3000x1500 with 4 Axes>